In [ ]:
import importlib
importlib.reload(crawler)

In [ ]:
hsc().crawl_n_pages()

,name,category,review,emojis,rating,url
0,Celimax The Vita-A Retinol Shot Tightening Serum,Сыворотка для лица,"Спасибо, что изменила мою жизнь",:pray:,5,https://hollyshop.ru/catalog/product/podtyagiv...
1,Celimax The Vita-A Retinol Shot Tightening Serum,Сыворотка для лица,"Отличная сыворотка, мне очень нравится эффект ...",NaN,5,https://hollyshop.ru/catalog/product/podtyagiv...
2,Celimax The Vita-A Retinol Shot Tightening Serum,Сыворотка для лица,NaN,:+1: :+1: :+1:,5,https://hollyshop.ru/catalog/product/podtyagiv...
3,Celimax The Vita-A Retinol Shot Tightening Serum,Сыворотка для лица,"Классная сыворотка, помогает при склонности ко...",NaN,5,https://hollyshop.ru/catalog/product/podtyagiv...
4,Celimax The Vita-A Retinol Shot Tightening Serum,Сыворотка для лица,Классная сыворотка. Реально работает. Кожа гла...,NaN,5,https://hollyshop.ru/catalog/product/podtyagiv...
...,...,...,...,...,...,...
15874,ETUDE My Beauty Tool Pimple Popper,Ложечка уно,"Хорошо убирает и черные точки, даже без исполь...",NaN,5,https://hollyshop.ru/catalog/product/lozhechka...
15875,ETUDE My Beauty Tool Pimple Popper,Ложечка уно,Очень удобно делать чистку)),NaN,5,https://hollyshop.ru/catalog/product/lozhechka...
15876,ETUDE My Beauty Tool Pimple Popper,Ложечка уно,"Приобрела ...особо еще не применяла, но качест...",NaN,5,https://hollyshop.ru/catalog/product/lozhechka...
15877,So Natural,Глиняная маска,Необычная и очень функциональная маска! После ...,NaN,5,https://hollyshop.ru/catalog/product/3d_maska_...


In [ ]:
MAX_HOLYSHOP = 100
holyshop_products = []
URL_HOLYSHOP = 'https://hollyshop.ru/catalog/basecare/?PAGEN_1='


for n in tqdm(range(1, MAX_HOLYSHOP + 1)):
    holyshop_n = requests.get(f'{URL_HOLYSHOP}{n}')
    holyshop_n = holyshop_n.content.decode('utf-8')
    soup_n = BeautifulSoup(holyshop_n)
    for div in soup_n.find_all('div', attrs={'class': 'catalog__list-item'}):
        if div.find('div', attrs={'class': 'tag-rating'}):
            link = f'https://hollyshop.ru{div.find('a').attrs['href']}reviews/'
            holyshop_products.append(link)
    time.sleep(5)

In [ ]:
with open('hollyshop_reviews.json', 'w', encoding='utf-8') as f:
    json.dump(holyshop_products, f, ensure_ascii=False, indent=4)

In [52]:
URL_CITILINK = 'https://www.citilink.ru/catalog/smartfony/?p='
MAX_CITILINK_PHONE = 17
citilink_products = []

In [53]:
for n in tqdm(range(1, MAX_CITILINK_PHONE + 1)):
    opts = Options()
    service = Service()
    driver = webdriver.Chrome(service=service, options=opts)

    try:
        driver.get(f'{URL_CITILINK}{n}')
        driver.implicitly_wait(10)
        citilink_n = driver.page_source
        soup_n = BeautifulSoup(citilink_n)
        for div in soup_n.find_all('div', attrs={'class': 'e1k9rx9o0'}):
            reviews = div.find('div', attrs={'class': 'app-catalog-gkt5l2-StyledMetaWrapper'})
            if reviews and reviews.find('a'):
                link = f'https://www.citilink.ru{reviews.find('a').attrs['href']}'
                citilink_products.append(link)
        time.sleep(5)
        
    finally:
        driver.quit()

  0%|          | 0/17 [00:00<?, ?it/s]

100%|██████████| 17/17 [05:11<00:00, 18.32s/it]


In [54]:
with open('citilink_reviews.json', 'w', encoding='utf-8') as f:
    json.dump(citilink_products, f, ensure_ascii=False, indent=4)

In [55]:
with open('hollyshop_reviews.json', 'r', encoding='utf-8') as f:
    holyshop_products = json.load(f)

In [86]:
holyshop_aggr_reviews = []


for url in tqdm(holyshop_products):
    holyshop_reviews = requests.get(url)
    holyshop_reviews = holyshop_reviews.content.decode('utf-8')
    soup_n = BeautifulSoup(holyshop_reviews)
    name = soup_n.find('div', attrs={'class': 'reviews-page__product-card-name'})
    if not name:
        continue
    category = soup_n.find('div', attrs={'class': 'reviews-page__product-card-category'}).text
    name = name.text
    for div in soup_n.find_all('div', attrs={'class': 'reviews__list-item'}):
        text = div.find('div', attrs={'class': 'reviews__list-item-body-content'}).text
        rating = div.find('div', attrs={'class': 'rating-stars'})
        rating = len(rating.find_all('span'))
        emojis = div.find_all('img', attrs={'class': 'emo'})
        if emojis:
            emojis = [f':{emoji.attrs['alt']}:' for emoji in emojis]
        holyshop_aggr_reviews.append(
            {
                'name': name,
                'category': category,
                'review': text.strip(),
                'emojis': ' '.join(emojis),
                'rating': rating,
                'url': url
            }
        )
    holyshop_df_reviews = pd.DataFrame(holyshop_aggr_reviews)
    holyshop_df_reviews.to_pickle('holyshop_df_reviews.pkl')
    time.sleep(1)

100%|██████████| 2854/2854 [58:15<00:00,  1.22s/it]  


In [87]:
holyshop_df_reviews = pd.DataFrame(holyshop_aggr_reviews)

In [89]:
holyshop_df_reviews.to_csv('hollyshop_reviews.csv', index=False)